In [65]:
class BigInt :
    def __init__(self, init = 0, sign = True, base = 2) :
        self.vec_rep = [] # list storing coefficients (private instance variab
        
        self.base = base
        
        # check type of sign
        if isinstance(sign, bool):
            self.sign = sign
        else:
            raise TypeError("sign should be of bool type")
        
        # check type of base
        if isinstance(base, int) and base > 1:
            self.base = base
        else:
            raise ValueError("base should be of positive integer larger than 1")
        
        # check type of init
        if isinstance(init, int) and init < base and init > -1:
            self.vec_rep = [init]
            
        elif isinstance(init, list) and all([-1<init[i]<base for i in range(len(init))]):
            self.vec_rep = init.copy()
        else:
            raise ValueError("init should be a list of non-negative integers less")
        
        #뒤에 불필요한 0이 생기는 현상이 있어, 이후 계산(특히 자릿수를 대소관계에 사용하는 sub)에 오류를 일으킬 수 있기에 지워주는 기능을 추가했습니다. 
        if len(self.vec_rep) != 0:
            while self.vec_rep[-1] == 0 and len(self.vec_rep) != 1:
                    (self.vec_rep).pop(-1)
                
    def length(self) :
            #return max([n for n,c in enumerate(self.vec_rep) if c != 0.0]+[0])+1
        return len(self.vec_rep)
    
    def get_integer(self):
        if self.sign == True:
            return sum(self.vec_rep[i]*pow(self.base,i) for i in range(self.length()))
        else:
            return -sum(self.vec_rep[i]*pow(self.base,i) for i in range(self.length()))
    
    def __neg__(self) :
        c = [self.vec_rep[i] for i in range(self.length())]
        return BigInt(init = c, sign = not self.sign, base = self.base)
    
    def __add__(self, adder) :
        if self.base != adder.base:
            return 'Bases {} and {} are different!'.format(self.base,adder.base)
        if self.sign == adder.sign:
            c = []
            # set a to be larger and b to be smaller
            if self.length() >= adder.length():
                a, b, k, l = self, adder, self.length(), adder.length()
            else:
                a, b, k, l = adder, self, adder.length(), self.length()
            B = self.base
            carry = 0
            for i in range(l):
                tmp = a.vec_rep[i] + b.vec_rep[i] + carry
                carry = tmp//B
                c.append(tmp % B)
            for i in range(l,k):
                tmp = a.vec_rep[i] + carry
                carry = tmp//B
                c.append(tmp % B)
            c.append(carry)
            return BigInt(init = c, sign = a.sign, base = B)
        else: # that is, self.sign != adder.sign:
            subter = - adder
            return self - subter
        
    def __sub__(self, subter) :
        if self.base != subter.base:
            return 'Bases {} and {} are different!'.format(self.base,subter.base)
             
        #뒤에 불필요한 0이 생기는 현상이 있어, 이후 계산(특히 자릿수를 대소관계에 사용하는 sub)에 오류를 일으킬 수 있기에 지워주는 기능을 추가했습니다.    
        if len(self.vec_rep) != 0:
            while self.vec_rep[-1] == 0 and len(self.vec_rep) != 1:
                (self.vec_rep).pop(-1)
                
        if len(subter.vec_rep) != 0:
            while subter.vec_rep[-1] == 0 and len(subter.vec_rep) != 1:
                (subter.vec_rep).pop(-1)

        if self.sign == subter.sign:
            # if signs of self and subter are the same, subtract from larger and d
            # First dertermine absolutely larger one
            self_len = self.length()
            sbt_len = subter.length()
            if self_len > sbt_len:
                larger, smaller, f_sign = self, subter, self.sign
            elif self_len == sbt_len:
                i = self_len - 1
                if self.vec_rep[i] > subter.vec_rep[i]:
                    larger, smaller, f_sign = self, subter, self.sign
                elif self.vec_rep[i] < subter.vec_rep[i]:
                    larger, smaller, f_sign = subter, self, not self.sign
                else:
                    while(self.vec_rep[i] == subter.vec_rep[i] and i > 0):
                        i -= 1
                        if self.vec_rep[i] > subter.vec_rep[i]:
                            larger, smaller, f_sign = self, subter, self.sign
                        elif self.vec_rep[i] < subter.vec_rep[i]:
                            larger, smaller, f_sign = subter, self, not self.sign
                    if i == 0 and (self.vec_rep[0] == subter.vec_rep[0]):
                        return BigInt(init = 0, sign = True, base = self.base) # t
            else:
                larger, smaller, f_sign = subter, self, not self.sign
            # If larger and smaller are determined, subtract smaller from larger
            c = []
            a, b, k, l = larger, smaller, larger.length(), smaller.length()
            B = self.base
            carry = 0
            for i in range(l):
                tmp = a.vec_rep[i] - b.vec_rep[i] + carry
                carry = tmp//B
                c.append(tmp % B)
            for i in range(l,k):
                tmp = a.vec_rep[i] + carry
                carry = tmp//B
                c.append(tmp % B)
            c.append(carry)

            
            #뒤에 불필요한 0이 생기는 현상이 있어, 이후 계산(특히 자릿수를 대소관계에 사용하는 sub)에 오류를 일으킬 수 있기에 지워주는 기능을 추가했습니다.    
            while c[-1] == 0:
                c.pop(-1)
                
            return BigInt(init = c, sign = f_sign, base = B)

        else: # that is self.sign() != subter.sign():
            adder = - subter
            return self + adder
        
    def __str__(self):
        result = ''
        if self.sign == True:
            result += '+('
        elif self.sign == False:
            result += '-('
        for i in range(self.length()):
            result += '{}'.format(self.vec_rep[i])
            if i != self.length()-1:
                result += ','
            else:
                result += ')_{}'.format(self.base)
        return result
    
    def __mul__(self, multiplier):
        if self.base != multiplier.base:
            return 'Bases {} and {} are different!'.format(self.base,multiplier.base)

        a, b, k, l, B = self, multiplier, self.length(), multiplier.length(), self.base
        c = [0 for i in range(k+l)]
            #c = []
            #for i in range(k+l):
                #c.append(0)
        for i in range(k):
            carry = 0
            for j in range(l):
                tmp = a.vec_rep[i] * b.vec_rep[j] + c[i+j] + carry
                carry, c[i+j] = tmp//B, tmp%B
            c[i+l] = carry
            
        #뒤에 불필요한 0이 생기는 현상이 있어, 이후 계산(특히 자릿수를 대소관계에 사용하는 sub)에 오류를 일으킬 수 있기에 지워주는 기능을 추가했습니다.    
        while c[-1] == 0:
            c.pop(-1)
            
        return BigInt(init = c, sign = (a.sign == b.sign), base = B)

    
    
    
    
    
    #----------------------------------------------------------------------------------------------------------------------------------------------------
    # 2019091676 경영학부 장진하
    # 09/27 전산수학 과제
    
    # <수정사항>
    # vec_rep 뒤에 불필요한 0이 생기는 현상이 있어, 이후 계산(특히 자릿수를 대소관계에 사용하는 sub)에 오류를 일으킬 수 있기에 지워주는 기능을 추가했습니다.
    # init의 인스턴스를 확인하는 과정에서 리스트의 각 값이 Base를 초과해도 통과되는 오류가 있어, 이를 해결하기 위해 and 뒤에 all을 추가했습니다.(29~31번줄)
    
    '''
    First Homework Assignment 1 / 3 :
    Write your own algorithm for division with remainder.
    If you need a reference, you can get some hint from [Shoup]. (But, not easy to read...)
    '''

    def __floordiv__(self,other):
        #elif isinstance(init, list) and all([-1<init[i]<base for i in range(len(init))]):
        #self.vec_rep = init.copy()
        #init의 인스턴스를 확인하는 과정에서 리스트의 각 값이 Base를 넘어가도 통과되는 오류가 있어, 
        #이를 해결하기 위해 and 뒤에 all을 추가했습니다.
        # one can remove the below error notification if the function supports div      
        if not isinstance(other, BigInt):
            raise TypeError("Input should be of BigInt type")

        if other.vec_rep == [0]:
            raise TypeError("Divisor can't be zero")


        self_tmp = BigInt(self.vec_rep.copy(), True, self.base)
        other_tmp = BigInt(other.vec_rep.copy(), True, self.base)

        i = 0

        while (self_tmp - (other_tmp * BigInt([0,1],True,self.base))).sign == True:
            other_tmp.vec_rep.insert(0,0)
            i += 1

        c = []

        for j in range(i+1):
            c.insert(0,0)
            while (self_tmp - other_tmp).sign == True:
                c[0] += 1
                self_tmp -= other_tmp
            other_tmp.vec_rep.pop(0)


        if self.sign == other.sign:

            if self.sign == True:#             (+++++++,++++++++)
                return BigInt(c, True, self.base)

            else: #self.sign == False:         (-------,--------)
                return BigInt(c, True, self.base)

        else:

            if self.sign == True:
                if self_tmp.vec_rep != [0]: #나머지 != 0        (+++++++,--------) 나머지 0 아님
                    #print(c)
                    return (BigInt(c, False, self.base) - BigInt([1],True,self.base))
                else:#                                        (+++++++,--------) 나머지 0
                    return BigInt(c, False, self.base)

            else: #self.sign == False:
                if self_tmp.vec_rep != [0]: #나머지 != 0        (-------,++++++++) 나머지 0 아님
                    #print(c)
                    return (BigInt(c, False, self.base) - BigInt([1],True,self.base))
                else:                       #                 (-------,++++++++) 나머지 0
                    return BigInt(c, False, self.base)

    
    
    def __mod__(self,other):
        if not isinstance(other, BigInt):
            raise TypeError("Input should be of BigInt type")

        #기존
        #elif isinstance(init, list) and all([-1<init[i]<base for i in range(len(init))]):
            #self.vec_rep = init.copy()
            #init의 인스턴스를 확인하는 과정에서 리스트의 각 값이 Base를 넘어가도 통과되는 오류가 있어, 
            #이를 해결하기 위해 and 뒤에 all을 추가했습니다.
        # one can remove the below error notification if the function supports div
        
        if other.vec_rep == [0]:
            raise TypeError("Divisor can't be zero")
        
        
        
        


        self_tmp = BigInt(self.vec_rep.copy(), True, self.base)
        other_tmp = BigInt(other.vec_rep.copy(), True, self.base)

        i = 0

        while (self_tmp - (other_tmp * BigInt([0,1],True,self.base))).sign == True:
            other_tmp.vec_rep.insert(0,0)
            i += 1

        c = []

        for j in range(i+1):
            c.insert(0,0)
            while (self_tmp - other_tmp).sign == True:
                c[0] += 1
                self_tmp -= other_tmp
            other_tmp.vec_rep.pop(0)


        if self.sign == other.sign:

            if self.sign == True:#             (+++++++,++++++++)
                return BigInt(self_tmp.vec_rep, True, self.base)

            else: #self.sign == False:         (-------,--------)
                return BigInt(self_tmp.vec_rep, False, self.base)

        else:

            if self.sign == True:
                if self_tmp.vec_rep != [0]: #나머지 != 0        (+++++++,--------) 나머지 0 아님
                    #print(c)
                    return (self_tmp - BigInt(other.vec_rep, True, self.base))
                else:#                                        (+++++++,--------) 나머지 0
                    return BigInt(self_tmp.vec_rep, False, self.base)

            else: #self.sign == False:
                if self_tmp.vec_rep != [0]: #나머지 != 0        (-------,++++++++) 나머지 0 아님
                    #print(c)
                    return (BigInt(other.vec_rep, True, self.base) - self_tmp)
                else:                       #                 (-------,++++++++) 나머지 0
                    return BigInt(self_tmp.vec_rep, True, self.base)
                
    '''                
    First Homework Assignment 2 / 3 
    Complete the functions, __truediv__.
    '''
    def __truediv__(self,other):
        if not isinstance(other, BigInt):
            raise TypeError("Input should be of BigInt type")
        # you may define your own code that takes less time than running two opera
        #기존
        #elif isinstance(init, list) and all([-1<init[i]<base for i in range(len(init))]):
            #self.vec_rep = init.copy()
            #init의 인스턴스를 확인하는 과정에서 리스트의 각 값이 Base를 넘어가도 통과되는 오류가 있어, 
            #이를 해결하기 위해 and 뒤에 all을 추가했습니다.
            
        # one can remove the below error notification if the function supports div
        
        if other.vec_rep == [0]:
            raise TypeError("Divisor can't be zero")
        
        
        
        


        self_tmp = BigInt(self.vec_rep.copy(), True, self.base)
        other_tmp = BigInt(other.vec_rep.copy(), True, self.base)

        i = 0

        while (self_tmp - (other_tmp * BigInt([0,1],True,self.base))).sign == True:
            other_tmp.vec_rep.insert(0,0)
            i += 1

        c = []

        for j in range(i+1):
            c.insert(0,0)
            while (self_tmp - other_tmp).sign == True:
                c[0] += 1
                self_tmp -= other_tmp
            other_tmp.vec_rep.pop(0)


        if self.sign == other.sign:

            if self.sign == True:#             (+++++++,++++++++)
                return BigInt(c, True, self.base), BigInt(self_tmp.vec_rep, True, self.base)

            else: #self.sign == False:         (-------,--------)
                return BigInt(c, True, self.base), BigInt(self_tmp.vec_rep, False, self.base)

        else:

            if self.sign == True:
                if self_tmp.vec_rep != [0]: #나머지 != 0        (+++++++,--------) 나머지 0 아님
                    #print(c)
                    return (BigInt(c, False, self.base) - BigInt([1],True,self.base)), (self_tmp - BigInt(other.vec_rep, True, self.base))
                else:#                                        (+++++++,--------) 나머지 0
                    return BigInt(c, False, self.base), BigInt(self_tmp.vec_rep, False, self.base)

            else: #self.sign == False:
                if self_tmp.vec_rep != [0]: #나머지 != 0        (-------,++++++++) 나머지 0 아님
                    #print(c)
                    return (BigInt(c, False, self.base) - BigInt([1],True,self.base)), (BigInt(other.vec_rep, True, self.base) - self_tmp)
                else:                       #                 (-------,++++++++) 나머지 0
                    return BigInt(c, False, self.base), BigInt(self_tmp.vec_rep, True, self.base)
                
                
                
                
                
                
        
        
'''
First Homework Assignment 3 / 3
Given a base and an integer , design a function that transfers from to a vector of digits in the base .
In particular, the result should be of list type, along with a bit indicating the sign.
'''
    
def get_vector(integer, base):
    init = []
    if integer > 0:
        while integer // base != 0:
            init.append(integer % base)
            integer = integer // base
        init.append(integer)
        return BigInt(init, True, base)
            
    elif integer == 0:
        return BigInt([0], True, base)
    
    else: #integer > 0:
        integer = -integer
        while integer // base != 0:
            init.append(integer % base)
            integer = integer // base
        init.append(integer)
        return BigInt(init, False, base)
    
